Terkadang data bervariatif dan kita ingin membuatnya menjadi seragam

In [1]:
def transform_value(row, mapping, column_name):
    data = row[column_name]
    value = None
    try:
        for map_ in mapping:
            if data in map_['from']:
                value = map_['to']

    except Exception as e:
        value = data
    
    if not value:
        value = data
    return value


In [2]:
mapping = [
            {
                    'from': ['L', 'laki-laki', 'l', 'Laki-laki'],
                    'to': 'laki-laki'
            },
            {
                    'from': ['P', 'perempuan',
                             'p', 'Perempuan'],
                    'to': 'perempuan'
            }]

In [3]:
import pandas as pd
import dask.dataframe as dd

In [4]:
df = pd.DataFrame()
df['gender'] = ['L', 'Laki-laki', 'laki-laki', 'l', 'laki', 'X', None, 'xman',
                'p', 'P', 'Perempuan', 'P', 'perempuan', None]
df = dd.from_pandas(df, npartitions=1)
df.head(10)

,gender
0,L
1,Laki-laki
2,laki-laki
3,l
4,laki
5,X
6,None
7,xman
8,p
9,P


In [5]:
column_name = 'gender'

df['new_col'] = df.apply(
    transform_value, args = [mapping, column_name], axis=1, meta=str)
df.head(20)

/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 20 elements requested, only 14 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,gender,new_col
0,L,laki-laki
1,Laki-laki,laki-laki
2,laki-laki,laki-laki
3,l,laki-laki
4,laki,laki
5,X,X
6,None,None
7,xman,xman
8,p,perempuan
9,P,perempuan


**Pendekatan ke dua**

In [6]:
similar_dict = {
    'L': 'laki-laki',
    'P': 'perempuan'
    }


In [10]:
def transform_value(row, similar_dict, column_name):
    keys = list(similar_dict.keys())
    data = str(row[column_name]).upper()
    
    if data == 'NONE':
        result = None
    else:
        sim_key = None
        for k in keys:
            if k in data:
                sim_key = k
                break
        if sim_key:
            result = similar_dict[sim_key]
        else:
            result = 'UNMAP_YET'

    return result

In [11]:
column_name = 'gender'
new_column_name = 'mapped_gender'
df[new_column_name] = df.apply(
    transform_value, args = [similar_dict, column_name], axis=1, meta=str)
df.head(100)

/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 100 elements requested, only 14 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,gender,new_col,mapped_gender
0,L,laki-laki,laki-laki
1,Laki-laki,laki-laki,laki-laki
2,laki-laki,laki-laki,laki-laki
3,l,laki-laki,laki-laki
4,laki,laki,laki-laki
5,X,X,UNMAP_YET
6,None,None,None
7,xman,xman,UNMAP_YET
8,p,perempuan,perempuan
9,P,perempuan,perempuan


Algoritma di atas memungkinkan kita untuk menemukan gender yang belum termap yakni 'UNMAP_YET', dengan begini kita dapat melakukan fiter gender apa saja yang belum termap

In [12]:
dfx = df[df['mapped_gender'] == 'UNMAP_YET']
dfx.head()

/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,gender,new_col,mapped_gender
5,X,X,UNMAP_YET
7,xman,xman,UNMAP_YET


In [13]:
unmap_yet = list(dfx['gender'])
unmap_yet

['X', 'xman']

Dengan begini kita dapat menambahkan ke similar_dict yang belum termap

In [14]:
similar_dict = {
    'L': 'laki-laki',
    'P': 'perempuan',
    'man': 'laki-laki',
    'X': 'laki-laki'
    }


Running fungsi yang sama di atas

In [15]:
column_name = 'gender'
new_column_name = 'additional_mapped_gender'
df[new_column_name] = df.apply(
    transform_value, args = [similar_dict, column_name], axis=1, meta=str)
df.head(100)

/Users/thomas/miniconda3/lib/python3.6/site-packages/dask/dataframe/core.py:5739: UserWarning: Insufficient elements for `head`. 100 elements requested, only 14 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,gender,new_col,mapped_gender,additional_mapped_gender
0,L,laki-laki,laki-laki,laki-laki
1,Laki-laki,laki-laki,laki-laki,laki-laki
2,laki-laki,laki-laki,laki-laki,laki-laki
3,l,laki-laki,laki-laki,laki-laki
4,laki,laki,laki-laki,laki-laki
5,X,X,UNMAP_YET,laki-laki
6,None,None,None,None
7,xman,xman,UNMAP_YET,laki-laki
8,p,perempuan,perempuan,perempuan
9,P,perempuan,perempuan,perempuan
